<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/prediction_upload/video_predictions.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/master/examples/prediction_upload/video_predictions.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Video Prediction Import 
* This notebook walks you through the process of uploading model predictions to a Model Run. This notebook provides an example for each supported prediction type for video assets.

A Model Run is a container for the predictions, annotations and metrics of a specific experiment in your ML model development cycle.

**Supported annotations that can be uploaded through the SDK**
- Bounding box
- Point
- Polyline
- Classification - radio
- Classification - checklist

**NOT** supported:
- Polygons 
- Segmentation masks
- Free form text classifications

Please note that this list of unsupported annotations only refers to limitations for importing annotations. For example, when using the Labelbox editor, segmentation masks can be created and edited on video assets.


## Setup

In [ ]:
!pip install -q 'labelbox[data]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import labelbox as lb
import uuid

## Replace with your API Key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
API_KEY = None
client = lb.Client(API_KEY)

In [ ]:
## Helper method - only required to update bbox_with_radio_subclass tool with nested classification

def set_feature_schema_id(features, nested_ndjson):
  """
  features: accespts a project's ontology features 
  nested_ndjson: nested ndjson annotation object
  name: tool name 

  """

  for i in features['tools']:
    print(i)
    if i['name'] == "bbox_with_radio_subclass":
      ## Classification feature schema id
      class_feature_schema_id = i['classifications'][0]['featureSchemaId']
      ## Answer feature schema id (select one of the answers)
      class_options_feature_schema_id = i['classifications'][0]['options'][0]['featureSchemaId']

      ## Update the original annotation with the schema ids
      for frame in nested_ndjson['segments']:
        for k in frame['keyframes']:
          k['classifications'][0].update(
              {'schemaId': class_feature_schema_id , 
                'answer': {'schemaId': class_options_feature_schema_id}
                }
              )
  return nested_ndjson

## Supported Predictions
- Only NDJSON annotations are supported with video assets
- Confidence scores are currently not supported for segment or frame annotations, which are required for bounding box, point, and line for video assets. For this tutorial, only the radio and checklist annotations will have confidence scores.

In [ ]:
########### Radio Classification ###########

# NDJSON
radio_prediction_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer', 'confidence': 0.5}
} 

In [ ]:
########## Nested Classification ##########

### Radio #### 

# NDJSON 
nested_radio_prediction_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer", 'confidence': 0.5 },
  "classifications" : [
   {
       'name': 'sub_radio_question', 
      'answer': {'name': 'first_sub_radio_answer', 'confidence': 0.5 }
    }
   ]
}

### Checklist #### 

nested_checklist_prediction_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{"name": "first_checklist_answer",
  'confidence': 0.5,
  "classifications" : [
   {
      "name": "sub_checklist_question", 
      "answer": {"name": "first_sub_checklist_answer", 'confidence': 0.5 }
    }
  ]
}]
}




In [ ]:
############ Checklist question ############

# NDJSON
checklist_prediction_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer', 'confidence': 0.5},
    {'name': 'second_checklist_answer', 'confidence': 0.5}
  ]
}

In [ ]:
####### Bounding box #######

#NDJSON 
bbox_prediction_ndjson = {
    "name" : "bbox_video",
    "segments" : [{
        "keyframes" : [
            {
              "frame": 1,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              } 
           },
           {
              "frame": 2,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           },
           {
              "frame": 3,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           },
           {
              "frame": 4,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           },
           {
              "frame": 5,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           }
        ]
      }
    ]
}


In [ ]:
####### Bounding box with nested classification #######

## NDJSON
#Confidence scores are not supported for bounding box, point, or line annotations,
#but they can be used for radio and checklist classifications, even when they're
#nested under a bounding box, point, or line annotation as demonstrated here.

bbox_with_radio_subclass_prediction_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "segments" : [{
      "keyframes" : [
          {
            "frame": 13,
            "bbox" : {
              "top": 146.0,
              "left": 98.0,
              "height": 382.0,
              "width": 341.0
            },
            "classifications": [{
            "schemaId": "",
            "answer": 
                {"schemaId": "", "confidence": 0.5} 
            }]  
          },
          {
            "frame": 14,
            "bbox" : {
              "top": 146.0,
              "left": 98.0,
              "height": 382.0,
              "width": 341.0
            },
            "classifications": [{
            "schemaId": "",
            "answer": 
                {"schemaId": "", "confidence": 0.5}
            }]  
          },
          {
            "frame": 15,
            "bbox" : {
              "top": 146.0,
              "left": 98.0,
              "height": 382.0,
              "width": 341.0
            },
            "classifications": [{
            "schemaId": "",
            "answer": 
                {"schemaId": "", "confidence": 0.5}
            }]
            }
          ]
      }
    ]
    

}

In [ ]:
######## Point ########

# NDJSON
point_prediction_ndjson = {
    "name": "point_video", 
    "segments": [{
        "keyframes": [{
            "frame": 17,
            "point" : {
                "x": 660.134 ,
                "y": 407.926
            },
        }]
    }] 
}

In [ ]:
###### Polyline ######

# NDJSON
polyline_prediction_ndjson = {
  "name": "line_video_frame", 
  "segments": [
      {
        "keyframes": [
          {
            "frame": 5,
            "line": [{
              "x": 680,
              "y": 100
            },{
              "x": 100,
              "y": 190
            },{
              "x": 190,
              "y": 220
            }],
          },
          {
            "frame": 12,
            "line": [{
              "x": 680,
              "y": 280
            },{
              "x": 300,
              "y": 380
            },{
              "x": 400,
              "y": 460
            }],
          },
          {
            "frame": 20,
            "line": [{
              "x": 680,
              "y": 180
            },{
              "x": 100,
              "y": 200
            },{
              "x": 200,
              "y": 260
            }],
          }
        ]
      },
      {
        "keyframes": [
          {
            "frame": 24,
            "line": [{
              "x": 300,
              "y": 310
            },{
              "x": 330,
              "y": 430
            }],
          },
          {
            "frame": 45,
            "line": [{
              "x": 600,
              "y": 810
            },{
              "x": 900,
              "y": 930
            }],
          }
        ]
      }
    ]
}


## Step 1: Import data rows into Catalog

In [ ]:
# send a sample image as batch to the project
test_img_url = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4",
    "global_key": str(uuid.uuid4())
}
dataset = client.create_dataset(name="video_prediction_demo")
data_row = dataset.create_data_row(test_img_url)
print(data_row)

<DataRow {
    "created_at": "2023-02-10 16:56:22+00:00",
    "external_id": null,
    "global_key": "95e90c5e-dc0f-45db-ac1a-ab6f7907cb0b",
    "media_attributes": {},
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4",
    "uid": "cldyrtalo008x07zof8jfeunx",
    "updated_at": "2023-02-10 16:56:22+00:00"
}>


## Step 2: Create/select an Ontology for your model predictions
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the name/instructions fields in your annotations to ensure the correct feature schemas are matched.


In [ ]:
ontology_builder = lb.OntologyBuilder(
  classifications=[ # List of Classification objects
    lb.Classification( # Radio classification given the name "text" with two options: "first_radio_answer" and "second_radio_answer"
      class_type=lb.Classification.Type.RADIO,
      name="radio_question", 
      options=[
        lb.Option(value="first_radio_answer"),
        lb.Option(value="second_radio_answer")
      ]
    ),
    lb.Classification( # Checklist classification given the name "text" with two options: "first_checklist_answer" and "second_checklist_answer"
      class_type=lb.Classification.Type.CHECKLIST,
      name="checklist_question", 
      options=[
        lb.Option(value="first_checklist_answer"),
        lb.Option(value="second_checklist_answer")
      ]
    ), 
    lb.Classification(
        class_type=lb.Classification.Type.RADIO,
        name="nested_radio_question",
        options=[
            lb.Option("first_radio_answer",
                options=[
                    lb.Classification(
                        class_type=lb.Classification.Type.RADIO,
                        name="sub_radio_question",
                        options=[lb.Option("first_sub_radio_answer")]
                    )
                ]
            )
          ] 
        ),
    lb.Classification(
      class_type=lb.Classification.Type.CHECKLIST,
      name="nested_checklist_question",
      options=[
          lb.Option(value="first_checklist_answer",
            options=[
              lb.Classification(
                  class_type=lb.Classification.Type.CHECKLIST,
                  name="sub_checklist_question", 
                  options=[lb.Option("first_sub_checklist_answer")]
              )
          ]
        )
      ]
    )      
  ],
  tools=[ # List of Tool objects
    lb.Tool( # Bounding Box tool given the name "box"
      tool=lb.Tool.Type.BBOX,
      name="bbox_video"), 
    lb.Tool( # Bounding Box tool given the name "box"
      tool=lb.Tool.Type.BBOX,
      name="bbox_with_radio_subclass",
      classifications=[
            lb.Classification(
                class_type=lb.Classification.Type.RADIO,
                name="sub_radio_question",
                options=[
                  lb.Option(value="first_sub_radio_answer")
                ]
              ),
        ]
      ),
 	  lb.Tool( # Point tool given the name "point"
      tool=lb.Tool.Type.POINT,
      name="point_video"), 
    lb.Tool( # Polyline tool given the name "line"
      tool=lb.Tool.Type.LINE,
      name="line_video_frame"),
    lb.Tool( # Relationship tool given the name "relationship"
      tool=lb.Tool.Type.RELATIONSHIP,
      name="relationship_video")]
)

ontology = client.create_ontology("Video Prediction Import Demo", ontology_builder.asdict(), media_type=lb.MediaType.Video)

## Required to run set_feature_schema_id helper method 
features_schema = ontology.normalized

## Step 3: Create a Model and Model Run

In [ ]:
# create Model
model = client.create_model(name="video_model_run_" + str(uuid.uuid4()), 
                            ontology_id=ontology.uid)
# create Model Run
model_run = model.create_model_run("iteration 1")

## Step 4: Send data rows to the Model Run

In [ ]:
model_run.upsert_data_rows([data_row.uid])

True

## Step 5. Create the predictions payload

Create the annotations payload using the snippets of [code here](https://docs.labelbox.com/reference/import-video-annotations).

Labelbox only supports NDJSON annotation payloads for importing video annotations.


In [ ]:
## Lets set the schema ids for the nested prediction
set_feature_schema_id(features_schema, bbox_with_radio_subclass_prediction_ndjson)

{'schemaNodeId': 'cldyrtcxi001707yra30a43ky', 'featureSchemaId': 'cldyrtcxi001607yr5j5b83ps', 'required': False, 'name': 'bbox_video', 'tool': 'rectangle', 'color': '#ff0000', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'cldyrtcxj001d07yrcpumdrzy', 'featureSchemaId': 'cldyrtcxi001807yrggaaal18', 'required': False, 'name': 'bbox_with_radio_subclass', 'tool': 'rectangle', 'color': '#7fff00', 'archived': 0, 'classifications': [{'schemaNodeId': 'cldyrtcxj001c07yr498u7sjy', 'featureSchemaId': 'cldyrtcxj001907yr3idhdbgi', 'archived': 0, 'required': False, 'instructions': 'sub_radio_question', 'name': 'sub_radio_question', 'type': 'radio', 'options': [{'schemaNodeId': 'cldyrtcxj001b07yr2afe29ej', 'featureSchemaId': 'cldyrtcxj001a07yr3ro1dhpb', 'label': 'first_sub_radio_answer', 'value': 'first_sub_radio_answer'}]}]}
{'schemaNodeId': 'cldyrtcxj001f07yrh4k3g2sl', 'featureSchemaId': 'cldyrtcxj001e07yrcge9h9b8', 'required': False, 'name': 'point_video', 'tool': 'point', 'color': '#00ff

{'name': 'bbox_with_radio_subclass',
 'segments': [{'keyframes': [{'frame': 13,
     'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0},
     'classifications': [{'schemaId': 'cldyrtcxj001907yr3idhdbgi',
       'answer': {'schemaId': 'cldyrtcxj001a07yr3ro1dhpb'}}]},
    {'frame': 14,
     'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0},
     'classifications': [{'schemaId': 'cldyrtcxj001907yr3idhdbgi',
       'answer': {'schemaId': 'cldyrtcxj001a07yr3ro1dhpb'}}]},
    {'frame': 15,
     'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0},
     'classifications': [{'schemaId': 'cldyrtcxj001907yr3idhdbgi',
       'answer': {'schemaId': 'cldyrtcxj001a07yr3ro1dhpb'}}]}]}]}

In [ ]:
# Create a Label object by identifying the applicable data row in Labelbox and providing a list of annotations
label_ndjson = []
for prediction in [
    radio_prediction_ndjson,
    checklist_prediction_ndjson,
    bbox_prediction_ndjson, 
    bbox_with_radio_subclass_prediction_ndjson,
    point_prediction_ndjson,
    polyline_prediction_ndjson, 
    nested_radio_prediction_ndjson,
    nested_checklist_prediction_ndjson
]:
  prediction.update({
      'dataRow': {'id': data_row.uid},
  })
  label_ndjson.append(prediction)

## Step 6. Upload the predictions payload to the Model Run 

In [ ]:
# Upload the prediction label to the Model Run
upload_job_prediction = model_run.add_predictions(
    name="prediction_upload_job"+str(uuid.uuid4()),
    predictions=label_ndjson)

# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_prediction.errors)

Errors: []


## Step 7: Send annotations to the Model Run 
To send annotations to a Model Run, we must first import them into a project, create a label payload and then send them to the Model Run.

##### 7.1. Create a labelbox project

In [ ]:
# Create a Labelbox project
project = client.create_project(name="video_prediction_demo",
                                    # Quality Settings setup 
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=lb.MediaType.Video)
project.setup_editor(ontology)

##### 7.2. Create a batch to send to the project 

In [ ]:
project.create_batch(
  "batch_video_prediction_demo", # Each batch in a project must have a unique name
  dataset.export_data_rows(), # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

<Batch ID: ebefc850-a963-11ed-b63a-8b5860515337>

##### 7.3 Create the annotations payload

In [ ]:
########### Radio Classification ###########

radio_annotation_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer'}
} 

########## Nested Classification ##########

### Radio #### 

nested_radio_annotation_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer"},
  "classifications" : [
   {
       'name': 'sub_radio_question', 
      'answer': {'name': 'first_sub_radio_answer'}
    }
   ]
}

### Checklist #### 

nested_checklist_annotation_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{"name": "first_checklist_answer",
  "classifications" : [
   {
      "name": "sub_checklist_question", 
      "answer": {"name": "first_sub_checklist_answer"}
    }
  ]
}]
}

############ Checklist question ############

checklist_annotation_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer'},
    {'name': 'second_checklist_answer'}
  ]
}

####### Bounding box #######

bbox_annotation_ndjson = {
    "name" : "bbox_video",
    "segments" : [{
        "keyframes" : [
            {
              "frame": 1,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              } 
           },
           {
              "frame": 2,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           },
           {
              "frame": 3,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           },
           {
              "frame": 4,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           },
           {
              "frame": 5,
              "bbox" : {
                "top": 146.0,
                "left": 98.0,
                "height": 382.0,
                "width": 341.0
              }
           }
        ]
      }
    ]
}

####### Bounding box with nested classification #######

bbox_with_radio_subclass_annotation_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "segments" : [{
      "keyframes" : [
          {
            "frame": 13,
            "bbox" : {
              "top": 146.0,
              "left": 98.0,
              "height": 382.0,
              "width": 341.0
            },
            "classifications": [{
            "schemaId": "",
            "answer": 
                {"schemaId":""}
            }]  
          },
          {
            "frame": 14,
            "bbox" : {
              "top": 146.0,
              "left": 98.0,
              "height": 382.0,
              "width": 341.0
            },
            "classifications": [{
            "schemaId": "",
            "answer": 
                {"schemaId":""}
            }]  
          },
          {
            "frame": 15,
            "bbox" : {
              "top": 146.0,
              "left": 98.0,
              "height": 382.0,
              "width": 341.0
            },
            "classifications": [{
            "schemaId": "",
            "answer": 
                {"schemaId":""}
            }]
            }
          ]
      }
    ]
}


 ######## Point ########

point_annotation_ndjson = {
    "name": "point_video", 
    "segments": [{
        "keyframes": [{
            "frame": 17,
            "point" : {
                "x": 660.134 ,
                "y": 407.926
            },
        }]
    }] 
}

###### Polyline ######

polyline_annotation_ndjson = {
  "name": "line_video_frame", 
  "segments": [
      {
        "keyframes": [
          {
            "frame": 5,
            "line": [{
              "x": 680,
              "y": 100
            },{
              "x": 100,
              "y": 190
            },{
              "x": 190,
              "y": 220
            }],
          },
          {
            "frame": 12,
            "line": [{
              "x": 680,
              "y": 280
            },{
              "x": 300,
              "y": 380
            },{
              "x": 400,
              "y": 460
            }],
          },
          {
            "frame": 20,
            "line": [{
              "x": 680,
              "y": 180
            },{
              "x": 100,
              "y": 200
            },{
              "x": 200,
              "y": 260
            }],
          }
        ]
      },
      {
        "keyframes": [
          {
            "frame": 24,
            "line": [{
              "x": 300,
              "y": 310
            },{
              "x": 330,
              "y": 430
            }],
          },
          {
            "frame": 45,
            "line": [{
              "x": 600,
              "y": 810
            },{
              "x": 900,
              "y": 930
            }],
          }
        ]
      }
    ]
}

##### 7.4. Create the label object

In [ ]:
## Lets set the schema ids for the nested annotation
set_feature_schema_id(features_schema, bbox_with_radio_subclass_annotation_ndjson)

{'schemaNodeId': 'cldyrtcxi001707yra30a43ky', 'featureSchemaId': 'cldyrtcxi001607yr5j5b83ps', 'required': False, 'name': 'bbox_video', 'tool': 'rectangle', 'color': '#ff0000', 'archived': 0, 'classifications': []}
{'schemaNodeId': 'cldyrtcxj001d07yrcpumdrzy', 'featureSchemaId': 'cldyrtcxi001807yrggaaal18', 'required': False, 'name': 'bbox_with_radio_subclass', 'tool': 'rectangle', 'color': '#7fff00', 'archived': 0, 'classifications': [{'schemaNodeId': 'cldyrtcxj001c07yr498u7sjy', 'featureSchemaId': 'cldyrtcxj001907yr3idhdbgi', 'archived': 0, 'required': False, 'instructions': 'sub_radio_question', 'name': 'sub_radio_question', 'type': 'radio', 'options': [{'schemaNodeId': 'cldyrtcxj001b07yr2afe29ej', 'featureSchemaId': 'cldyrtcxj001a07yr3ro1dhpb', 'label': 'first_sub_radio_answer', 'value': 'first_sub_radio_answer'}]}]}
{'schemaNodeId': 'cldyrtcxj001f07yrh4k3g2sl', 'featureSchemaId': 'cldyrtcxj001e07yrcge9h9b8', 'required': False, 'name': 'point_video', 'tool': 'point', 'color': '#00ff

{'name': 'bbox_with_radio_subclass',
 'segments': [{'keyframes': [{'frame': 13,
     'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0},
     'classifications': [{'schemaId': 'cldyrtcxj001907yr3idhdbgi',
       'answer': {'schemaId': 'cldyrtcxj001a07yr3ro1dhpb'}}]},
    {'frame': 14,
     'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0},
     'classifications': [{'schemaId': 'cldyrtcxj001907yr3idhdbgi',
       'answer': {'schemaId': 'cldyrtcxj001a07yr3ro1dhpb'}}]},
    {'frame': 15,
     'bbox': {'top': 146.0, 'left': 98.0, 'height': 382.0, 'width': 341.0},
     'classifications': [{'schemaId': 'cldyrtcxj001907yr3idhdbgi',
       'answer': {'schemaId': 'cldyrtcxj001a07yr3ro1dhpb'}}]}]}]}

In [ ]:
# Create a Label object by identifying the applicable data row in Labelbox and providing a list of annotations

ndjson_annotation = []
for annot in [
    radio_annotation_ndjson, 
    checklist_annotation_ndjson, 
    bbox_annotation_ndjson, 
    point_annotation_ndjson, 
    polyline_annotation_ndjson,
    nested_radio_annotation_ndjson,
    nested_checklist_prediction_ndjson,
    bbox_with_radio_subclass_annotation_ndjson
]:
  annot.update({
      'dataRow': {'id': data_row.uid},
  })
  ndjson_annotation.append(annot) 

##### 7.5. Upload annotations to the project using Label Import

In [ ]:
upload_job_annotation = lb.LabelImport.create_from_objects(
    client = client,
    project_id = project.uid,
    name="video_annotations_import_" + str(uuid.uuid4()),
    labels=ndjson_annotation)

upload_job_annotation.wait_until_done()
# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_annotation.errors)


            Confidence scores are not supported in Label Import.
            Corresponding confidence score values will be ignored.
            


Errors: []


##### 7.6. Send the annotations to the Model Run

In [ ]:
# get the labels id from the project
label_ids = [x['ID'] for x in project.export_labels(download=True)]
model_run.upsert_labels(label_ids)

True

## Optional deletions for cleanup 


In [ ]:
# project.delete()
# dataset.delete()